# XGBoost原理与应用

作者：杨岱川

时间：2019年11月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

参考文章：

- [XGBoost PPT](https://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf)
- [XGBoost: A Scalable Tree Boosting System](https://arxiv.org/abs/1603.02754)
- [Introduction to Boosted Trees](https://xgboost.readthedocs.io/en/latest/tutorials/model.html)
- [XGBoost原理](https://www.zhihu.com/question/58883125/answer/206813653)
- [XGBoost原理及目标函数推导详解](https://blog.csdn.net/htbeker/article/details/91517805)

## XGBoost

在[2.10 提升方法](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/02机器学习基础/2.10%20提升方法.ipynb)中我们简单提到过，在SKLearn之外还有很多优秀的，基于Boosting Tree方法的衍生框架。包括XGBoost，LightGBM等，而各种开源比赛方案对它们的应用，也从事实上证明了这些框架的优秀性能。

因此这一节会简要介绍XGBoost这一新框架的原理和用法。

### 简介

**XGBoost** 全称为“eXtreme Gradient Boosting”，是一种基于决策树的集成机器学习算法，使用梯度提升框架，适用于分类和回归问题。XGBoost 在各种比赛及比赛平台中大放异彩，比如Kaggle，天池，DataCastle等等，可以说是所有选手的必备武器之一。XGBoost 的项目主页参见[这里](https://xgboost.ai)。

XGBoost 最初由陈天奇开发。陈天奇本科毕业于上海交通大学 ACM 班，博士毕业于华盛顿大学计算机系，研究方向为大规模机器学习，2020年他将加入 CMU 出任助理教授。有兴趣的可以看看他的文章[《陈天奇：机器学习科研的十年》](http://www.sohu.com/a/328234576_129720)以及[访谈](https://cosx.org/2015/06/interview-of-tianqi)，也可以看看他的[知乎主页](https://www.zhihu.com/people/crowowrk/activities)。

下图展示了从决策树到 XGBoost 算法的发展过程：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/xgboost%E5%8F%91%E5%B1%95%E5%8E%86%E7%A8%8B.jpeg?raw=true)

图片刷不出来的请看[这里](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/xgboost发展历程.jpeg)

在 [XGBoost.ai](https://xgboost.ai) 官网有对 XGBoost原理有通俗易懂的解释。

现在简要介绍如下，部分基础知识之前也已经讨论过了，现在就当复习。其中的各种符号跟这里和论文保持一致，所以和之前的 GBDT 介绍可能有少许差别。

### 目标函数：经验损失 + 正则化

训练模型的目标是找到最好的参数 $\theta$ ，能够最佳拟合训练数据 $x_i$ 和标签 $y_i$。所以XGBoost为了训练模型，定义了一个**目标函数（objective function）** 来告诉模型如何去拟合训练数据。

目标函数由两部分组成：**经验损失**（又叫**训练损失**，**training loss**）和**正则项（regularization term）**。

$$\large {\rm obj}(\theta)=L(\theta)+\Omega(\theta)$$

其中， $L$ 就是训练损失，而 $\Omega$ 就是正则项。

通常，$L$ 会选择均方误差，如下：

$$\large L(\theta)=\sum_i(y_i-\hat y_i)^2$$

其中，$\hat y_i=\sum_j \theta_j x_{ij}$，也就是模型的预测值。

另一种常见的损失函数叫做**logistic loss**，也就是之前在逻辑回归中讲过的损失函数：

$$\large L(\theta) = \sum_i \left[ y_i \ln(1+e^{-\hat y_i})+(1-y_i)\ln(1+e^{-\hat y_i}) \right]$$

至于正则项$\Omega$，就是控制模型复杂度的，让模型不至于过拟合，以前的章节讲过多次，这里不再赘述。

### 决策树的集成

XGBoost模型是基于决策树的集成（**decision tree ensembles**）而来，这个集成模型是由一组**分类与回归树（CART）**所构成的，CART 的分裂基于 gini 系数，关于 CART 的详细描述可以参见《统计学习方法》中决策树一章。

这里给了一个简单的例子，比如我们要运用 CART 来判断某人是否喜欢电脑游戏。我们输入一家人的年龄、性别、职业等信息，然后得到一颗决策树：

![img](https://raw.githubusercontent.com/dmlc/web-data/master/xgboost/model/cart.png)

图片刷不出来的看这里：[CART](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/cart.png)

我们将家庭成员分为不同的叶子结点（leaves），然后在相应的叶子上给他们分配分数（score）。

>**score** 怎么来的？
>
>由于这里使用的是 CART，所以当作是回归树而不要当作分类树来看，叶子结点上给出的是回归预测值，而非分类预测值。
>
>在每个叶子结点采用这种真实分数，可以提供超越“分类”的更丰富的解释，比如给出分类概率等，也会更方便优化，在后面会有展示。

之前有提到，一棵决策树树是不够强大的，不管是随机森林、AdaBoost还是GBDT，都是集成模型，我们需要把多个**弱分类器**（基本分类器）的预测结果汇总在一起，形成一个**强分类器**，这就是**集成模型（ensemble model）**。

在这里，我们汇总的方法就是把多棵树的 score 加起来，如图：

![img](https://raw.githubusercontent.com/dmlc/web-data/master/xgboost/model/twocart.png)

图片刷不出来的看这里：[two CART](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/twocart.png)

以上图两棵 CART 树为例，可以看出模型的集成就是简单地把两棵树叶子结点对实例的预测分数相加即可。上面这个例子表达了一个重要的点，就是两棵 CART 互为补充，以数学的形式展示就是:

$$\large \hat y_i=\sum^K_{k=1}f_k(x_i),\quad f_k\in \Bbb F$$

其中 $K$ 是树的数量， $f$ 就是一棵CART树的函数，属于函数空间 $\Bbb F$，而 $\Bbb F$ 是所有可能的 CART 的集合。

我们待优化的目标函数在一开始已经给出了，这里对其稍作细化：

$$\large {\rm obj}(\theta)=\sum_i^n l(y_i,\hat y_i) + \sum^K_{k=1}\Omega(f_k)$$

左边的 $\sum_i^n l(y_i,\hat y_i)$ 是损失函数，右边的 $\sum^K_{k=1}\Omega(f_k)$ 是正则项。

现在的回顾一下，随机森林中使用的 *模型* 是也是集成的树（Tree ensembles），所以 boosted trees 和 random forests 的核心原理还是一样的，只不过训练过程中有点点区别。这也是我们在讲Boost算法的时候叙述过的。

### 树的Boosting

如何训练模型？相信大家肯定不陌生，老样子，那就是 *定义目标函数然后对其优化* ！

将下式作为目标函数。需要说明的是，记住它始终包含了正则项，而在我们 2.10节中讲的 AdaBoost 和 GBDT，是没有涉及正则项的。

$$\large {\rm obj}= \sum_{i=1}^n l(y_i,\hat y_i^{(t)}) + \sum^t_{i=1}\Omega(f_k)$$

#### Additive Training

第一个问题：树的 **参数** 是什么？我们要学习的是那些 $f_i$ 函数，每个函数都包含树的结构和叶子得分（leaf scores）。

在这里针对树的训练，采用加法策略（前向分步算法），也就是和之前 Boosting 讲的一样，根据学习结果进行权重修正，然后新增一棵树。

我们定义在时间步 $t$ 时候的预测值为 $\large \hat y_i^{(t)}$，有:

$$
\large
\begin{equation}\begin{split} 
\hat y_i^{(0)}&=0 \\
\hat y_i^{(1)}&=f_1(x_i)=\hat y_i^{(0)}+f_1(x_i)\\ 
\hat y_i^{(2)}&=f_1(x_i)+f_2(x_i)=\hat y_i^{(1)}+f_2(x_i)\\ 
\cdots\\ 
\hat y_i^{(t)}&=\sum^t_{k-1}f_k(x_i)=\hat y_i^{(t-1)}+f_t(x_i)
\end{split}\end{equation}
$$

还有一个问题就是，我们在每一步要的是那棵树？我们自然要选择能优化我们**目标函数**的那棵。

在 $t$ 步的时候，目标函数 ${\rm obj}^{(t)}$ 为：

$$
\large
\begin{equation}\begin{split} 
{\rm obj}^{(t)} &=  \sum_{i=1}^n l(y_i,\hat y_i^{(t)}) + \sum^t_{i=1}\Omega(f_i)\\
&= \sum_{i=1}^n l(y_i,\hat y_i^{(t-1)}+f_t({x_i})) + \Omega(f_t)+\sum^{t-1}_{k=1}\Omega(f_t)\\
&= \sum_{i=1}^n l(y_i,\hat y_i^{(t-1)}+f_t({x_i})) + \Omega(f_t)+{\rm constant}
\end{split}\end{equation}
$$

> 这个 $\rm constant$ 就是常数的意思。由于前向分步算法，在 $t$ 步时候，前 $t-1$ 步的树都是已经确定了的，因此其结构是一个常数。
>
> 相当于 $\large \sum^{t-1}_{k=1}\Omega(f_t)={\rm constant}$

如果我们使用**均方误差（mean squared error ，MSE）**作为我们的损失函数，即替换 $l(y_i,\hat y_i^{(t)})$，那么目标函数就变成了：

$$
\large
\begin{equation}\begin{split} 
{\rm obj}^{(t)} &=  \sum_{i=1}^n \left(y_i - \left(\hat y_i^{(t-1)}+f_t(x_i)\right)\right)^2 + \sum^t_{i=1}\Omega(f_i)\\
&= \sum_{i=1}^n \left[ 2\left( \hat y_i^{(t-1)}-y_i \right)f_t(x_i)+f_t(x_i)^2 \right] + \Omega(f_t)+{\rm constant}
\end{split}\end{equation}
$$

> 注意这里 $\large \hat y_i^{(t)}$ 实际上就是 $t-1$ 步之前的 CART 森林加上第 $t$ 步生成的 CART 树的集成模型的预测结果，即:
>
> $$\large \hat y_i^{(t)} = \hat y_i^{(t-1)}+f_t(x_i)$$
>
>故均方误差为：
>
> $$\large \left(y_i-\hat y_i^{(t)}\right)^2$$

均方误差的表示形式非常方便，存在一次项和二次项。但是如果损失函数换为其他损失，比如 logistic loss，要获得这样的直观形式就比较困难了。所以，在一般情况下，我们将损失函数的**泰勒展开式（Taylor expansion）**扩展到二阶：

$$\large {\rm obj}^{(t)}=\sum_{i=1}^n \left[l(y_i,\hat y_i^{(t-1)}) + g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i)\right] + \Omega(f_t)+{\rm constant}$$

其中 $g_i$ 和 $h_i$ 定义为：

$$\large g_i=\partial_{\hat y_i^{(t-1)}}l(y_i,\hat y_i^{(t-1)})$$

$$\large h_i=\partial^2_{\hat y_i^{(t-1)}}l(y_i,\hat y_i^{(t-1)})$$

就是对损失函数求预测值 $\large \hat y_i^{(t-1)}$ 的一阶和二阶偏导数。

> 泰勒公式的二阶近似可以表示为：
>
>$$\large f(x_0+\Delta x) \approx f(x_0)+f'(x_0)\Delta x+\frac{1}{2}f''(x_0)(\Delta x)^2$$
>
> 这里的增量 $\Delta x$ 相当于 $t-1$ 步的森林中新增一棵 $t$ 时间步的树，也就是 $f_t(x_i)$
>
> 至于为什么要求偏导数，因为 $\hat y_i^{(t)}$ 是我们要优化的目标，而不是 $y_i$，可以看看**梯度下降**相关内容。

现在我们去掉所有常数项，因为在优化过程中保留常数没啥必要，目标函数在 $t$ 步时候就变成了：

$$\large \sum^n_{i=1}\left[ g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i) \right] + \Omega(f_t)$$

这就是我们对新的树的优化目标。

我们新优化目标的优点在于目标函数的取值仅仅取决于 $g_i$ 和 $h_i$，也就是损失函数的一二阶导数，所以 XGBoost 支持自定义损失函数，只要它二阶可导。

> XGBoost 的特点就是将损失函数泰勒展开到了二阶，GBDT只用到了一阶，可以回头看看[2.10 提升方法](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/%E6%9D%AD%E7%94%B5%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E8%AF%BE%E7%A8%8B%E5%8F%8A%E4%BB%A3%E7%A0%81/2.10%20%E6%8F%90%E5%8D%87%E6%96%B9%E6%B3%95.ipynb)中求负梯度那部分。

### 模型复杂度

刚刚已经介绍过了训练的过程，但是还没有涉及非常重要的**正则项（regularization term）**，所以这里我们先定义树的复杂度 $\Omega(f)$。

首先将树的函数 $f(x)$ 细化为：

$$\large f_t(x)=w_{q(x)},\quad w \in R^T,q:R_d \rightarrow \{1,2,\cdots,T\}$$

其中 $w$ 是叶子上分数的矢量，$q$ 是将每个数据实例分配给相应叶子结点的函数，$T$ 是叶子的总数。

换句话说，$q(x)$ 就是输出的叶子节点的序号，$w_{q(x)}$ 表示对应的叶子节点的得分。

在 XGBoost中，我们将复杂度定义为：

$$\large \Omega(f)=\gamma T+\frac{1}{2}\lambda\sum^T_{j=1}w^2_j$$

相当于**叶子的总数** $T$ 加上 **叶子分数的L2正则**，$j$ 是叶子结点的编号。

定义模型复杂度的方法很多，但是这种定义方法在实践中表现的很好。在以前的很多基于决策树的工具中不是很在意这部分，一般都是把复杂度的控制留给了一些启发式算法。在 XGBoost 中给出了一个正式的定义，我们可以更好地了解模型的训练过程。

### The Structure Score

定义了模型的复杂度之后，我们将模型的目标函数重写：

$$
\large
\begin{equation}\begin{split} 
{\rm obj}^{(t)} &\approx  \sum_{i=1}^n \left[ g_iw_{q(x_i)}+\frac{1}{2}h_iw^2_{q(x_i)} \right] + \gamma T+\frac{1}{2}\lambda\sum^T_{j=1}w^2_j \\
&= \sum_{j=1}^T \left[ \left( \sum_{i\in I_j}g_i \right)w_j + \frac{1}{2}\left( \sum_{i\in I_j}h_i + \lambda \right)w_j^2 \right] + \gamma T
\end{split}\end{equation}
$$

其中，$I_j=\{ i|q(x_i)=j \}$，表示分配给第 $j$ 个叶子的样本 $x_i$ 的索引，换句话说就是回归树中的叶结点区域。

然后上式把所有的样本点做了一下合并，因为同一个叶子结点上所有样本的分数都是一样的。

定义：

$$\large G_j=\sum_{j\in I_j}g_i$$

$$\large H_j=\sum_{j\in I_j}h_i$$

我们可以进一步简化目标函数 ${\rm obj}^{(t)}$：

$$\large {\rm obj}^{(t)}=\sum^T_{j=1}\left[ G_jw_j + \frac{1}{2}(H_j+\lambda)w_j^2 \right]+\gamma T$$

在这个等式中，$w_j$ 是彼此独立的，$G_jw_j + \cfrac{1}{2}(H_j+\lambda)w_j^2$ 是关于 $w_j$ 的一元二次函数，对于给定的 $q(x)$，对 $w_j$ 求导可以得到最优的 $w_j$ 取值：

$$\large w_j^*=-\frac{G_j}{H_j+\lambda}$$

把上式代入回 ${\rm obj}^{(t)}$：

$$\large {\rm obj}^*=-\frac{1}{2}\sum^T_{j=1}\frac{G_j^2}{H_j+\lambda}+\gamma T$$

得到最终的目标函数 ${\rm obj}^*$，它也称为**打分函数（scoring function）**，用以衡量树结构 $q(x)$ 的好坏，值越小代表结构越好。我们采用这个打分函数来选择 CART 的最佳切分点。

![img](https://raw.githubusercontent.com/dmlc/web-data/master/xgboost/model/struct_score.png)

图片刷不出来的看这里：[struct_score](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/struct_score.png)

如图，基本上对于给定的树结构，我们将统计实例的一二阶导数，也就是梯度信息 $g_i$ 和 $h_i$ ，然后放入它们所对应的叶子结点。再对叶子上实例的梯度信息求和，使用上面推出的公式计算树的质量。其实有点类似于计算一棵 CART 决策树的 gini 不纯度，但是不同之处在于它还考虑的模型的复杂度。

关于**gini不纯度**可以参考《统计学习方法》中决策树一章对 CART 的讲解。

### 学习树的结构

到目前为止，我们有了一种衡量一棵决策树质量的方法，理想情况下，我们会列举出所有可能的决策树，然后选择最好的一棵。但是实际上这是几乎不可能的。所以我们会试着一次优化树的一层（level），层层深入。

具体来说，我们把一个结点切分为两个子结点，其得分为：

$$\large Gain=\frac{1}{2}\left[ \frac{G^2_L}{H_L+\lambda}+\frac{G^2_R}{H_R+\lambda}-\frac{(G_L+G_R)^2}{H_L+H_R+\lambda} \right]-\gamma$$

该公式可以理解为新切分的左叶子结点的分数加上新切分的右叶子结点的分数减去原来叶子结点的分数，外加一个叶上的正则化。

这里有一个重要的事实，就是如果新切分的增益小于 $\gamma$，就不要添加该分支，这相当于是决策树的剪枝技术。这个 $\gamma$ 值是一个超参数，人工设定。

我们希望搜索到一个最佳的切分，而为了达成这个目标，将所有实例按照排序顺序放置，如图,按照年龄排序：

![img](https://raw.githubusercontent.com/dmlc/web-data/master/xgboost/model/split_find.png)

然后我们从左到右开始扫描分裂点，计算这些 CART 树的得分，就可以高效地找到最佳拆分点。

> 前向算法的局限性
>
> 在少部分边缘情况下，前向算法可能会失败，扩展阅读：[Can Gradient Boosting Learn Simple Arithmetic?](http://mariofilho.com/can-gradient-boosting-learn-simple-arithmetic/)

### 结语

到现在我们已经完成了对 XGBoost 原理的学习，而 XGBoost 也根据以上原理编写成了 XGBoost 工具，记得尝试一下这一款优秀的机器学习工具吧！

## XGBoost 安装和使用

参考[Get Started with XGBoost](https://xgboost.readthedocs.io/en/latest/get_started.html)

示例代码请先参考官方文档，本 NoteBook 待补充。